# A Closed Form Reduced Order Electrochemical Model for Lithium-Ion Cells

## Introduction

This model considers a Li-ion cell connected to an external load in a circuit. We consider that that a 3-dimensional cylindrical cell can be opened up as a sheet of paper of negligible thickness, now also assume that no change in any physical quantity occurs along the width. This turn our 3D system to a 1D one. 

Now according to standard conventions let's assume that our cell consists of **3 regions**, the **-ve electrode**, **the separator** and the **+ve electrode**. Also assume that the cell has collectors at end of either electrode. We'll be having the standard reactions in the +ve and -ve electrodes as below:

*-ve electrode:*

<div align=center>LiC₆ → x.Li⁺ + Li₁₋ₓC₆ + x.e⁻</div>

*+ve electrode:*

<div align=center>x.Li⁺ + LiMO₂ + x.e⁻ → Li₁₊ₓMO₂</div>


With this our system should look like below.

<div align=center><img src="../assets/senthil/li_cell.png"></div>

Assume that we have `x = 0` at the -ve electrode and `x = L` at the +ve electrode, L being the cell length. Length of the regions from left to right being lₙ, lₛ and lₚ respectively.

## The difference
This model build upon the SenthilModel where the reaction rate was assumed constant. However, that assumption is not accurate as cell reaction rate changes with time, to accomodate for the above condition, it is assusmed that the cell reaction rate follows an exponentially decreasing profile. This model is also called the Non uniform reduced order model, henceforth **NUROM**.


## Importing necessary libraries
To get started import the CaseStudy's senthil model (as sm for convinence) as that's the model which we're simulating. Also import Gadfly as that's the plotting library we're gonna use. To see the model constants [check here.](../src/ashwini/constants/constants.jl)

In [ ]:
import LiionBatteryModels.AshwiniModel as am 
using Gadfly

set_default_plot_size(800px, 600px)

## Cell Voltage for constant current
In this example we'll be showing the cases of *constant current*. The example describes all the internal model variables also.

In [ ]:
I = 13.5
t = Float64.(0:1:25000)
tspan = 10

In [ ]:
# Start model solving
dₚ₁ᵢ = am.dₚ₁₀(I)
dₙ₁ᵢ = am.dₙ₁₀(I)

0.06088358249081125

In [4]:
# Reaction rates @x = 0, x = L
jₙ = am.jₙ(0, dₙ₁ᵢ, dₚ₁ᵢ, I)
jₚ = am.jₚ(0, dₙ₁ᵢ, dₚ₁ᵢ, I)

-2.2396633194924437e-6

In [5]:
# Storing required time series of variables
# c₂ᵢₙ = zeros(size(t)[1])
# c₂ᵢₚ = zeros(size(t)[1])

c₂̄ₙ = zeros(size(t)[1])
c₂̄ₚ = zeros(size(t)[1])
c₂̄ₛ = zeros(size(t)[1])

c₂ₙ₀ = zeros(size(t)[1])
c₂ₚₗ = zeros(size(t)[1])

jₙ  = zeros(size(t)[1])
jₚ = zeros(size(t)[1])



25001-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [6]:
c₂̄ₙ[1] = am.c₂₀
c₂̄ₚ[1] = am.c₂₀
c₂̄ₛ[1] = am.c₂₀

c₂ₙ₀[1] = am.c₂₀
c₂ₚₗ[1] = am.c₂₀

jₙ[1] = am.jₙ(0, dₙ₁ᵢ, dₚ₁ᵢ, I)
jₚ[1] = am.jₚ(am.L, dₙ₁ᵢ, dₚ₁ᵢ, I)

-1.982895967118508e-6

In [7]:
for i = 2:convert(Int, t[end])
    params = am.param_solver(I, tspan, c₂̄ₙ[i-1], c₂̄ₚ[i-1], c₂̄ₛ[i-1], c₂ₙ₀[i-1], c₂ₚₗ[i-1], jₙ[i-1], jₚ[i-1])

    n0 = params[1]; n2 = params[3]; n3 = params[4];
    p0 = params[5]; p2 = params[7]; p3 = params[8];
    s0 = params[9]; s2 = params[10]; s3 = params[11];

    c₂̄ₙ[i] = params[12];
    c₂̄ₚ[i] = params[13];
    c₂̄ₛ[i] = params[14];
    c₂ₙ₀[i] = params[15];
    c₂ₚₗ[i] = params[16];

    println(params)
end

[1.9256865931784206e10; 0.0; 4.0118470604234373e18; -4.457607845023648e22; -2.2519168890717976e10; 0.0; -3.0025558569242414e18; 2.6689385395891846e22; 1.9256865931784206e10; -1.718766310861989e15; 1.087439389261943e19; 40.34271663540188; 38.769614260744504; 7604.426026608209; 39.873053522742445; 39.62155718180604;;]
[-6602.959930456766; 0.0; 2.168106100079497e11; 7.646135391263785e14; 5661.028180998834; 0.0; -1.1813969136742833e9; 2.2498886256752774e11; -6602.959930456766; 4.940108687162674e8; -2.8403755000360244e12; -3530.638927672663; 5643.876730352094; -897.02219973437; -2159.6349507319082; 5635.206087852327;;]
[1213.89091681204; 0.0; -5.338782209350428e7; -1.0336653274740135e10; 1220.1890086512988; 0.0; -2.0516785968060741e9; -2.1892337612861057e12; 1213.89091681204; 236398.30424049404; -8.820636532846651e8; 1213.3649974172981; 1183.8723317409522; 1216.9076531836765; 1213.1042704370345; 1166.6375762788216;;]
[1.491122187978987e11; 0.0; 3.106504557392936e19; -3.451671730453988e23; -

Excessive output truncated after 524291 bytes.

; -7.83417531960913e19; 40.76533479057718; 39.313878078678286; 7601.719852714504; 40.014130084540305; 38.76953889831314;;]
[-6636.899536408365; 0.0; 2.1796308566621353e11; 7.71532482971938e14; 5675.907043122585; 0.0; -1.1819867275894644e9; 2.2444596012798315e11; -6636.899536408365; 4.9607363617734075e8; -2.854891673099193e12; -3544.547816081083; 5658.7453710453165; -908.7299976077693; -2165.0229722393806; 5650.069846867253;;]
[1213.3254726949897; 0.0; -5.266267769349929e7; -1.0262520244850458e10; 1219.7306372857545; 0.0; -1.9909612359780672e9; -2.1264633315330085e12; 1213.3254726949897; 240418.786037949; -8.971099893044022e8; 1212.8066107628947; 1184.4836084381404; 1216.3934884919763; 1212.54939650122; 1167.7571957323244;;]
[-1.0186672064441283e11; 0.0; -2.122223347654828e19; 2.358025941821309e23; 1.119678433764637e11; 0.0; 1.4929045778068937e19; -1.3270262913903563e23; -1.0186672064441283e11; 8.137929553557e15; -3.3670358428703863e19; 40.76533479057718; 39.313878078678286; 7601.719852